In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MiAplicacion") \
    .getOrCreate()

In [2]:
df = spark.read.parquet("data/final/merge_jugadores.parquet")
df.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- Born: float (nullable = true)
 |-- MP: float (nullable = true)
 |-- Starts: float (nullable = true)
 |-- Min: float (nullable = true)
 |-- Gls: double (nullable = true)
 |-- Ast: double (nullable = true)
 |-- YellowC: float (nullable = true)
 |-- RedC: float (nullable = true)
 |-- PrgC: double (nullable = true)
 |-- PrgP: double (nullable = true)
 |-- PrgR: double (nullable = true)
 |-- Fls: double (nullable = true)
 |-- Off: double (nullable = true)
 |-- Crosses: double (nullable = true)
 |-- Recov: double (nullable = true)
 |-- Aerialwon%: float (nullable = true)
 |-- Tack_Def_3rd: double (nullable = true)
 |-- Tack_Mid_3rd: double (nullable = true)
 |-- Tack_Att_3rd: double (nullable = true)
 |-- Tkl%: float (nullable = true)
 |-- Tkl: double (nullable = true)
 |-- TklW: double (nullable = true)
 |-- Int: double (nullab

In [57]:
from datetime import date
from pyspark.sql.functions import col, expr
def calcular_edad(df): 
    df = df.withColumn("Edad", date.today().year - col("Born"))
    return df

df=calcular_edad(df)

In [79]:
position_metrics = {
    'Centre-Back': [
        'Tkl', 'Int', 'Blocks', 'Clearences', 'Aerialwon%',
        'Touch_Def_3rd', 'Tack_Def_3rd', 'Pass_cmp%', 'PrgC', 'PrgP',
        'Tkl%', 'Block_Shots', 'Pass_cmp_Short%', 'Pass_Short',
        'Pass_cmp_Medium%', 'Pass_Medium', 'Pass_cmp_Long%', 'Pass_Long',
        'PrgDist'
    ],
    'Left-Back': [
        'PrgC', 'PrgP', 'Crosses', 'Recov', 'Tack_Def_3rd', 'Tkl%', 'Tkl', 'Int',
        'Block_Shots', 'Bolck_Pass', 'Clearences', 'Pass_cmp', 'Pass_cmp%',
        'Pass_cmp_Short%', 'Pass_Short', 'Pass_cmp_Medium%', 'Pass_Medium', 'xA',
        'A-xAG', 'CrsPA', 'Touch_Def_3rd', 'Touch_Att_3rd', 'drib_Att', 'drib_Succ%',
        'PrgDist', 'Carries_Att_3rd', 'Touch_Live'
    ],
    'Right-Back': [
        'PrgC', 'PrgP', 'Crosses', 'Recov', 'Tack_Def_3rd', 'Tkl%', 'Tkl', 'Int',
        'Block_Shots', 'Bolck_Pass', 'Clearences', 'Pass_cmp', 'Pass_cmp%',
        'Pass_cmp_Short%', 'Pass_Short', 'Pass_cmp_Medium%', 'Pass_Medium', 'xA',
        'A-xAG', 'CrsPA', 'Touch_Def_3rd', 'Touch_Att_3rd', 'drib_Att', 'drib_Succ%',
        'PrgDist', 'Carries_Att_3rd', 'Touch_Live'
    ],
    'Defensive Midfield': [
        'Tkl', 'Int', 'Blocks', 'Clearences', 'Aerialwon%', 'Touch_Def_3rd',
        'Touch_Mid_3rd', 'Tack_Def_3rd', 'Pass_cmp', 'Pass_cmp%', 'PrgC', 'PrgP',
        'Tkl%', 'Block_Shots', 'Pass_cmp_Short%', 'Pass_cmp_Long%',
        'Pass_cmp_Medium%', 'PrgDist', 'Recov',
        'Tack_Mid_3rd', 'Bolck_Pass', 'Touch_Live'
    ],
    'Central Midfield': [
        'Ast', 'PrgC', 'PrgP', 'Recov', 'Aerialwon%', 'Tack_Def_3rd', 'Tack_Mid_3rd',
        'Tack_Att_3rd', 'Tkl%', 'Tkl', 'Int', 'Bolck_Pass', 'Pass_cmp',
        'Pass_cmp%', 'Pass_cmp_Short%', 'Pass_Short', 'Pass_cmp_Medium%', 'Pass_Medium',
        'Pass_Long', 'Pass_cmp_Long%', 'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA',
        'Touch_Def_3rd', 'Touch_Mid_3rd', 'Touch_Att_3rd', 'Tckl_Drib', 'Tckl_Drib%',
        'PrgDist', 'Carries_Att_3rd', 'Touch_Live'
    ],
    'Attacking Midfield': [
        'Ast', 'PrgC', 'PrgP', 'PrgR', 'Tack_Att_3rd', 'Tack_Mid_3rd', 'Bolck_Pass',
        'Pass_cmp', 'Pass_cmp%', 'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium',
        'Pass_cmp_Medium%', 'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA',
        'Touch_Mid_3rd', 'Touch_Att_3rd', 'Touch_Att_Pen', 'drib_Att', 'drib_Succ%',
        'Tckl_Drib', 'Tckl_Drib%', 'PrgDist', 'Carries_Att_3rd', 'Carries_Att_Pen',
        'Sh', 'SoT', 'G/Sh', 'Touch_Live'
    ],
    'Right Midfield': [
        'Gls', 'Ast', 'PrgC', 'PrgP', 'PrgR', 'Crosses', 'Tack_Mid_3rd', 'Tack_Att_3rd',
        'Tkl%', 'Tkl', 'Bolck_Pass', 'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium',
        'Pass_cmp_Medium%', 'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA', 'CrsPA',
        'Touch_Mid_3rd', 'Touch_Att_3rd', 'drib_Att', 'drib_Succ%', 'Tckl_Drib',
        'Tckl_Drib%', 'PrgDist', 'Carries_Att_3rd', 'Carries_Att_Pen',
        'Sh', 'SoT', 'G/Sh'
    ],
    'Left Midfield': [
        'Gls', 'Ast', 'PrgC', 'PrgP', 'PrgR', 'Crosses', 'Tack_Mid_3rd', 'Tack_Att_3rd',
        'Tkl%', 'Tkl', 'Bolck_Pass', 'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium',
        'Pass_cmp_Medium%', 'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA', 'CrsPA',
        'Touch_Mid_3rd', 'Touch_Att_3rd', 'drib_Att', 'drib_Succ%', 'Tckl_Drib',
        'Tckl_Drib%', 'PrgDist', 'Carries_Att_3rd', 'Carries_Att_Pen',
        'Sh', 'SoT', 'G/Sh'
    ],
    'Left Winger': [
        'Gls', 'Ast', 'PrgC', 'PrgP', 'PrgR', 'Crosses', 'Tack_Att_3rd', 'Tkl%', 'Tkl',
        'Bolck_Pass', 'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium', 'Pass_cmp_Medium%',
        'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA', 'CrsPA', 'Touch_Att_3rd',
        'Touch_Att_Pen', 'drib_Att', 'drib_Succ%', 'Tckl_Drib', 'Tckl_Drib%',
        'PrgDist', 'Carries_Att_3rd', 'Carries_Att_Pen',
        'Sh', 'SoT', 'G/Sh', 'xG', 'npxG', 'npxG/Sh', 'G-xG'
    ],
    'Right Winger': [
        'Gls', 'Ast', 'PrgC', 'PrgP', 'PrgR', 'Crosses', 'Tack_Att_3rd', 'Tkl%', 'Tkl',
        'Bolck_Pass', 'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium', 'Pass_cmp_Medium%',
        'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA', 'CrsPA', 'Touch_Att_3rd',
        'Touch_Att_Pen', 'drib_Att', 'drib_Succ%', 'Tckl_Drib', 'Tckl_Drib%',
        'PrgDist', 'Carries_Att_3rd', 'Carries_Att_Pen', 
        'Sh', 'SoT', 'G/Sh', 'xG', 'npxG', 'npxG/Sh', 'G-xG'
    ],
    'Second Striker': [
        'Gls', 'Ast', 'PrgC', 'PrgP', 'PrgR', 'Tack_Att_3rd', 'Bolck_Pass',
        'Pass_cmp_Short%', 'Pass_Medium', 'Pass_cmp_Medium%', 'xAG', 'xA', 'A-xAG',
        'Pass_cmp_Att_3rd', 'PPA', 'Touch_Mid_3rd', 'Touch_Att_3rd', 'Touch_Att_Pen',
        'drib_Att', 'drib_Succ%', 'Tckl_Drib', 'Tckl_Drib%', 'PrgDist',
        'Carries_Att_3rd', 'Carries_Att_Pen', 
        'Sh', 'SoT', 'G/Sh', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'Touch_Live'
    ],
    'Centre-Forward': [
        'Gls', 'Ast', 'PrgR', 'Aerialwon%', 'Tack_Att_3rd', 'Tkl%', 'Tkl', 'Bolck_Pass',
        'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium', 'Pass_cmp_Medium%', 'xAG',
        'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'Touch_Att_3rd', 'Touch_Att_Pen',
        'Carries_Att_Pen', 'drib_Att', 'drib_Succ%', 
        'Sh', 'SoT', 'G/Sh', 'xG', 'npxG', 'npxG/Sh', 'G-xG'
    ]
}

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import col, expr, log1p, expm1
def train_models_by_position_rf_extended(df, position_metrics, market_col="market_value", min_minutes=600):
    results = []

    # Variables adicionales
    extra_numeric = ["Edad", "MP"]
    extra_categorical = ["Competition"]
    for position, metrics in position_metrics.items():
        print(f"Entrenando delo RF para: {position}")

        # Filtrar por posición y minutos
        df_pos = df.filter((col("Pos") == position) & (col("Min") >= min_minutes))

        # Eliminar nulos
        all_cols = metrics + extra_numeric + extra_categorical + [market_col]
        df_pos = df_pos.dropna(subset=all_cols)

        # Cast numéricos
        for colname in metrics + extra_numeric:
            df_pos = df_pos.withColumn(colname, col(colname).cast("double"))

         # Asegurar market_value válido
        df_pos = df_pos.withColumn("market_value", col(market_col).cast("double"))
        df_pos = df_pos.filter(col("market_value") > 0)

        # Log-transformación
        df_pos = df_pos.withColumn("log_market_value", log1p(col("market_value")))

        # Categóricas: indexar + one hot
        indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep") for c in extra_categorical]
        encoders = [OneHotEncoder(inputCol=f"{c}_idx", outputCol=f"{c}_vec") for c in extra_categorical]

        # Ensamblar todas las features
        all_features = metrics + extra_numeric + [f"{c}_vec" for c in extra_categorical]
        assembler = VectorAssembler(inputCols=all_features, outputCol="features")
        
        # Modelo Random Forest
        rf = RandomForestRegressor(
            featuresCol="features",
            labelCol="log_market_value",
            predictionCol="predicted_log_value",
            numTrees=100,
            maxDepth=8
        )

        # Pipeline completo
        pipeline = Pipeline(stages=indexers + encoders + [assembler, rf])
        model = pipeline.fit(df_pos)
        df_pred = model.transform(df_pos)

        from pyspark.sql.functions import expm1
        df_pred = df_pred.withColumn("predicted_value", expm1(col("predicted_log_value")))

        # Value gap
        df_pred = df_pred.withColumn("value_gap", col("predicted_value") - col("market_value"))
        df_pred = df_pred.withColumn("value_gap_pct", (col("value_gap") / col("market_value")) * 100)
        df_pred = df_pred.withColumn("modelo_posicion", expr(f"'{position}'"))

        # Selección final
        df_final_pos = df_pred.select(
            "Player", "Pos", "Edad", "Competition", "MP",
            "market_value", "predicted_value", "value_gap", "value_gap_pct",
            "modelo_posicion", *metrics
        )

        results.append(df_final_pos)

    # Unión final
    df_final = results[0]
    for df_r in results[1:]:
        df_final = df_final.unionByName(df_r, allowMissingColumns=True)

    return df_final


In [58]:
df.describe().show()

+-------+--------------------+---------+---------+-----------+------------------+------------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+-------------------+------------------+-----------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-----------------+------------------+-------------------+-------------------+--------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-------------------

In [80]:
df_final=train_models_by_position(df, position_metrics)

Entrenando modelo para: Centre-Back
Entrenando modelo para: Left-Back
Entrenando modelo para: Right-Back
Entrenando modelo para: Defensive Midfield
Entrenando modelo para: Central Midfield
Entrenando modelo para: Attacking Midfield
Entrenando modelo para: Right Midfield
Entrenando modelo para: Left Midfield
Entrenando modelo para: Left Winger
Entrenando modelo para: Right Winger
Entrenando modelo para: Second Striker
Entrenando modelo para: Centre-Forward


In [62]:
from pyspark.ml.evaluation import RegressionEvaluator

# Evaluador sobre log(valor mercado)
evaluator_r2 = RegressionEvaluator(
    labelCol="log_market_value", 
    predictionCol="predicted_value", 
    metricName="r2"
)

evaluator_rmse = RegressionEvaluator(
    labelCol="log_market_value", 
    predictionCol="predicted_value", 
    metricName="rmse"
)

r2 = evaluator_r2.evaluate(df_final)
rmse = evaluator_rmse.evaluate(df_final)

print(f"R² (log): {r2:.4f}")
print(f"RMSE (log): {rmse:.4f}")


IllegalArgumentException: [FIELD_NOT_FOUND] No such struct field `log_market_value` in `Player`, `Pos`, `market_value_scaled`, `predicted_value_real`, `predicted_value`, `value_gap`, `value_gap_pct`, `modelo_posicion`, `Tkl`, `Int`, `Blocks`, `Clearences`, `Aerialwon%`, `Touch_Def_3rd`, `Tack_Def_3rd`, `Pass_cmp%`, `PrgC`, `PrgP`, `Tkl%`, `Block_Shots`, `Pass_cmp_Short%`, `Pass_Short`, `Pass_cmp_Medium%`, `Pass_Medium`, `Pass_cmp_Long%`, `Pass_Long`, `PrgDist`, `Crosses`, `Recov`, `Bolck_Pass`, `Pass_cmp`, `xA`, `A-xAG`, `CrsPA`, `Touch_Att_3rd`, `drib_Att`, `drib_Succ%`, `Carries_Att_3rd`, `Touch_Live`, `Touch_Mid_3rd`, `Tack_Mid_3rd`, `Ast`, `Tack_Att_3rd`, `xAG`, `Pass_cmp_Att_3rd`, `PPA`, `Tckl_Drib`, `Tckl_Drib%`, `PrgR`, `Touch_Att_Pen`, `Carries_Att_Pen`, `Sh`, `SoT`, `G/Sh`, `Gls`, `xG`, `npxG`, `npxG/Sh`, `G-xG`. SQLSTATE: 42704

In [73]:
df_final.show()

+--------------------+-----------+-------------------+--------------------+------------------+--------------------+--------------------+---------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-------+-----+----------+--------+----+-----+-----+-------------+--------+----------+---------------+----------+-------------+------------+----+------------+----+----------------+----+---------+----------+----+-------------+---------------+----+----+----+----+----+----+-------+----+
|              Player|        Pos|market_value_scaled|predicted_value_real|   predicted_value|           value_gap|       value_gap_pct|modelo_posicion|                Tkl|               Int|       

In [40]:
df_final.select("Player", "market_value_scaled", "predicted_value_real", "Min")

{"ts": "2025-07-31 22:01:14.287", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Min` cannot be resolved. Did you mean one of the following? [`Int`, `Sh`, `xA`, `xG`, `Ast`]. SQLSTATE: 42703", "context": {"file": "jdk.internal.reflect.GeneratedMethodAccessor17.invoke(Unknown Source)", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o29879.select.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Min` cannot be resolved. Did you mean one of the following? [`Int`, `Sh`, `xA`, `xG`, `Ast`]. SQLSTATE: 42703;\n'Project [Player#0, market_value_scaled#45398, predicted_value_real#45692, 'Min]\n+- Union false, false\n   :- Project [Player#0, Pos#67, market_value_scaled#45398, predicted_value_re

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Min` cannot be resolved. Did you mean one of the following? [`Int`, `Sh`, `xA`, `xG`, `Ast`]. SQLSTATE: 42703;
'Project [Player#0, market_value_scaled#45398, predicted_value_real#45692, 'Min]
+- Union false, false
   :- Project [Player#0, Pos#67, market_value_scaled#45398, predicted_value_real#45692, predicted_value#45688, value_gap#45689, value_gap_pct#45690, modelo_posicion#45691, Tkl#45379, Int#45380, Blocks#45381, Clearences#45382, Aerialwon%#45383, Touch_Def_3rd#45384, Tack_Def_3rd#45385, Pass_cmp%#45386, PrgC#45387, PrgP#45388, Tkl%#45389, Block_Shots#45390, Pass_cmp_Short%#45391, Pass_Short#45392, Pass_cmp_Medium%#45393, Pass_Medium#45394, Pass_cmp_Long%#45395, ... 34 more fields]
   :  +- Union false, false
   :     :- Project [Player#0, Pos#67, market_value_scaled#45398, predicted_value_real#45692, predicted_value#45688, value_gap#45689, value_gap_pct#45690, modelo_posic